# The University of Melbourne, School of Computing and Information Systems
# COMP30027 Machine Learning, 2019 Semester 1
-----
## Project 1: Gaining Information about Naive Bayes
-----
###### Student Name(s): Xinyao Niu
###### Python version: 3.6.8
###### Submission deadline: 1pm, Fri 5 Apr 2019

This iPython notebook is a template which you may use for your Project 1 submission. (You are not required to use it; in particular, there is no need to use iPython if you do not like it.)

Marking will be applied on the five functions that are defined in this notebook, and to your responses to the questions at the end of this notebook.

You may change the prototypes of these functions, and you may write other functions, according to your requirements. We would appreciate it if the required functions were prominent/easy to find. 

In [138]:
import numpy as np
import os
from os.path import join
import subprocess
import pandas as pd
from collections import defaultdict as dd
from math import log

In [411]:
def bash_call(c:[str]) -> None:
    res = subprocess.check_output(c)
    ls = []
    for line in res.splitlines():
        line = line.decode('utf-8')
        if line.split('.')[-1] != "txt":
            ls.append(line)
    return ls

In [413]:
datapath = join(os.getcwd(),'2019S1-proj1-data')
files = bash_call(['ls', datapath])
print(files)

['anneal.csv', 'breast-cancer.csv', 'car.csv', 'cmc.csv', 'hepatitis.csv', 'hypothyroid.csv', 'mushroom.csv', 'nursery.csv', 'primary-tumor.csv']


In [147]:
datapath

'/Users/xinyaoniu/Documents/COMP30027-ML/prject 1/2019S1-proj1-data'

In [148]:
HEADERS = {
        "anneal.csv": "family,product-type,steel,carbon,hardness,temper_rolling,condition,formability,strength,non-ageing,surface-finish,surface-quality,enamelability,bc,bf,bt,bw-me,bl,m,chrom,phos,cbond,marvi,exptl,ferro,corr,bbvc,lustre,jurofm,s,p,shape,oil,bore,packing,class".split(","),
        "breast-cancer.csv": "age,menopause,tumor-size,inv-nodes,node-caps,deg-malig,breast,breast-quad,irradiat,class".split(","),
        "car.csv": "buying,maint,doors,persons,lug_boot,safety,class".split(","),
        "cmc.csv": "w-education,h-education,n-child,w-relation,w-work,h-occupation,standard-of-living,media-exposure,class".split(","),
        "hepatitis.csv": "sex,steroid,antivirals,fatigue,malaise,anorexia,liver-big,liver-firm,spleen-palpable,spiders,ascites,varices,histology,class".split(","),
        "hypothyroid.csv": "sex,on-thyroxine,query-on-thyroxine,on_antithyroid,surgery,query-hypothyroid,query-hyperthyroid,pregnant,sick,tumor,lithium,goitre,TSH,T3,TT4,T4U,FTI,TBG,class".split(","),
        "mushroom.csv": "cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,stalk-shape,stalk-root,stalk-surface-above-ring,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat,class".split(","),
        "nursery.csv": "parents,has_nurs,form,children,housing,finance,social,health,class".split(","),
        "primary-tumor.csv": "age,sex,histologic-type,degree-of-diffe,bone,bone-marrow,lung,pleura,peritoneum,liver,brain,skin,neck,supraclavicular,axillar,mediastinum,abdominal,class".split(","),
    }

In [424]:
class naive_bayes_learner():
    
    def __init__(self, path:str, file:str):
        self.file = file
        self.path = join(path, file)
        self.df = None
        self.labels = []  
        self.attributes = []  # possible attributes for each row
        self.last = -1
        
        self.prob = []
        self.label_prob = []
        self.test_size = -1
        self.train_size = -1
        self.train_set = pd.DataFrame()
        self.test_set = pd.DataFrame()
        
        self.predictions = []
        
        self.ig = {}
        
        
    def preprocess(self, val:str='hold-out', shuffle:bool=False, hold:float=0.2) -> None:
        """
        open a csv and transform it into a usable format
        
        *val -- define the evaluation method, it use hold out in defualt
        
        *shuffle -- whether we want to shuffle the dataset, it set to false in defaut

        *hold -- if the val method is hold-out, this parameter can change the proportion between
        training set and test set, it set to 0.2 in default
        """
        self.df = pd.read_csv(self.path, header=None)
        self.df.columns = HEADERS[self.file]
        self.size = len(self.df.index)
        # labels location
        self.last = self.df.columns[-1]
        # remove duplicate labels
        self.labels = list(set(self.df[self.last]))
        
        if shuffle:
            self.df = self.df.sample(frac=1).reset_index(drop=True)
        
        if((val=="hold-out") and (hold != 0)):
            self.test_size = int(self.df.shape[0] * hold)
            self.train_size = self.df.shape[0] - self.test_size

            self.train_set = self.df.head(self.train_size)
            self.test_set = self.df.tail(self.test_size)
        else:
            self.train_set = self.df
            self.test_set = self.df
            self.test_size = self.df.shape[0]
            self.train_size = self.df.shape[0]
        
        print("train=", self.train_set.shape, "test=", self.test_set.shape)

        # pre-process the train set
        for i in self.train_set.columns:
            stats = self.count_frequency(self.train_set[[i,self.last]])
            self.attributes.append(tuple(set(self.train_set[i])))
            self.prob.append(stats)
            #print("len of prob =", len(self.prob))
        #print("len of header =", len(self.train_set.columns))
            
    
    def count_frequency(self, df:pd.DataFrame ,accumulate:dict={}, laplace:bool=False) -> dd:
        """
        helper function for preprocess
        counting the frequency for each different value in an array
        """
        if(laplace):
            res = dd(lambda:1, accumulate)
        else:
            res = dd(lambda:0,accumulate)
        
        index = df.columns
        same = False
        
        # rename the index if they are the same
        if index[0] == index[1]:
            index = ['one', 'two']
            df.columns = index
            same = True
            
        for i in range(df.shape[0]):
            feature = df[index[0]].loc[i]
            label = df[index[1]].loc[i]
            if same:
                res[str(label)] += 1
            elif str(feature) == '?': 
                #ignore the missing value
                pass
            else:
                res[str(feature)+'|'+ str(label)] += 1
        return res
    
    
    def find_condition(self, prob:str, label, diag) -> str:
        """
        Helper function of train
        finding the condition of a conditional probability
        """
        try:
            return prob.split("|")[1]
        except:
            print(prob)
            print(label)
            print(diag)
            print(self.prob)
            return prob.split('|')[1]
    
    
    def train(self) -> None:
        """
        Calculating the conditional probability for NB
        """
        label = self.prob[-1]
        for i in range(len(self.prob)-1):
            current = self.prob[i]
            #print(current)
            for k,v in current.items():
                current[k] /= label[self.find_condition(k, label, ["i=",i,"cols=", len(self.df.columns), "prob=",len(self.prob)])]
        
        for k,v in label.items():
            label[k] /= self.train_size
        
        self.prob[-1] = label
    
    
    def predict(self) -> None:
        """
        predicting the class for an instance or a set of instances, basd on a trained model
        """
        
        features = self.test_set[self.test_set.columns[:-1]]
        results = self.test_set[self.test_set.columns[-1]]
        
        for i in features.index:
            p = np.zeros(len(self.labels))
            for l in range(len(self.labels)):
                p[l] = (self.prob_calculator(features.loc[i], self.labels[l]))
            #print(features.loc[i])
            #print(self.labels)
            #print(p, self.labels[np.argmax(p)], results.loc[i])
            self.predictions.append((self.labels[np.argmax(p)], results.loc[i]))
    
    
    def prob_calculator(self, features:pd.DataFrame, target:str) -> float:
        """
        helper function of predict
        calculate the probability based on the given features
        """
        res = log(self.prob[-1][target],2)
        n = 0
        for f in features:
            if str(f) != '?':
                condition = str(f) + '|' + target
                # using epsilon smoothing in default
                if (self.prob[n][condition] <= 0):
                    res += log(0.001,2)
                else:
                    res += log(self.prob[n][condition],2)
            n += 1
        return res


    def evaluate(self) -> None:
        """
        evaluate a set of predictions, in a supervised context
        """
        #print(self.predictions)
        total = len(self.predictions)
        correct = 0
        
        for pair in self.predictions:
            if pair[0] == pair[1]:
                correct += 1
        
        print("correct =", correct, "total =",total)
        print("percison =",correct/total)
        
    
    def info_gain(self) -> None:
        """
        calculate the information gain of an attributes with respect to the labels
        """
        full = []  # contains the frequency of each atttribute along their column
        for i in self.test_set:
            full.append(self.count_frequency(self.df[[i,self.last]]))
        
        parent = 0
        n = self.df.shape[0]
        
        for i in full[-1].values():
            parent += self.entropy(i/n)
        #print(parent, n)
        #print(full[-1])
        
        for i in range(len(full)-1):
            current = full[i]
            child = 0
            for attr in self.attributes[i]:
                a = []
                for l in self.labels:
                    # ignore the missing value
                    if attr != "?":
                        target = str(attr) + "|" + l
                        #print(target, current[target])
                        a.append(current[target])
                #print(a,child,"p=",a[0]/sum(a))
                for n in a:
                    child += (n/self.size)*self.entropy(n/sum(a))
            #print(self.df.columns[i])
            self.ig[self.df.columns[i]] = parent-child
        
        

    def entropy(self, p:float) -> float:
        """
        helper function for info_gain
        mainly for calculate the entropy for given probability
        
        *p -- given probability
        """
        if(p==0):
            return 0
        return -p*log(p,2)
        
        
    
    def run(self, v="hold-out", h=0.2, s=False, l=False) -> None:
        """
        this function will automatically run through the workflow with different
        set up parameters. It is only for convenient testing and debug purpose.
        """
        self.preprocess(shuffle=s, val=v, hold=h)
        
        #print(self.prob[-1])
        #print("=====================")
        #print(self.attributes)
        
        self.train()
        
        print(list(range(len(self.prob)-1)))
        #print("=====================")
        #print(len(self.prob)-1)
        #print(self.prob[len(self.prob)-1])
        #print("=====================")
        
        self.predict()
        self.evaluate()
        
        #self.info_gain()
        #print(self.ig)
        

In [426]:
for i in files:
    print(i)
    test = naive_bayes_learner(datapath, i)
    test.run(v='hold-out',s=True)
    print("---------------------------------------------")
    test = naive_bayes_learner(datapath, i)
    test.run(v='hold-out',s=False)
    print("---------------------------------------------")
    test = naive_bayes_learner(datapath, i)
    test.run(v='full',s=False)
    print("=============================================")

anneal.csv
train= (719, 36) test= (179, 36)
len of prob = 1
len of prob = 2
len of prob = 3
len of prob = 4
len of prob = 5
len of prob = 6
len of prob = 7
len of prob = 8
len of prob = 9
len of prob = 10
len of prob = 11
len of prob = 12
len of prob = 13
len of prob = 14
len of prob = 15
len of prob = 16
len of prob = 17
len of prob = 18
len of prob = 19
len of prob = 20
len of prob = 21
len of prob = 22
len of prob = 23
len of prob = 24
len of prob = 25
len of prob = 26
len of prob = 27
len of prob = 28
len of prob = 29
len of prob = 30
len of prob = 31
len of prob = 32
len of prob = 33
len of prob = 34
len of prob = 35
len of prob = 36
len of header = 36
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34]
correct = 173 total = 179
percison = 0.9664804469273743
---------------------------------------------
train= (719, 36) test= (179, 36)
len of prob = 1
len of prob = 2
len of prob = 3
len of prob = 4
len 

KeyboardInterrupt: 

Questions (you may respond in a cell or cells below):

1. The Naive Bayes classifiers can be seen to vary, in terms of their effectiveness on the given datasets (e.g. in terms of Accuracy). Consider the Information Gain of each attribute, relative to the class distribution — does this help to explain the classifiers’ behaviour? Identify any results that are particularly surprising, and explain why they occur.
2. The Information Gain can be seen as a kind of correlation coefficient between a pair of attributes: when the gain is low, the attribute values are uncorrelated; when the gain is high, the attribute values are correlated. In supervised ML, we typically calculate the Infomation Gain between a single attribute and the class, but it can be calculated for any pair of attributes. Using the pair-wise IG as a proxy for attribute interdependence, in which cases are our NB assumptions violated? Describe any evidence (or indeed, lack of evidence) that this is has some effect on the effectiveness of the NB classifier.
3. Since we have gone to all of the effort of calculating Infomation Gain, we might as well use that as a criterion for building a “Decision Stump” (1-R classifier). How does the effectiveness of this classifier compare to Naive Bayes? Identify one or more cases where the effectiveness is notably different, and explain why.
4. Evaluating the model on the same data that we use to train the model is considered to be a major mistake in Machine Learning. Implement a hold–out or cross–validation evaluation strategy. How does your estimate of effectiveness change, compared to testing on the training data? Explain why. (The result might surprise you!)
5. Implement one of the advanced smoothing regimes (add-k, Good-Turing). Does changing the smoothing regime (or indeed, not smoothing at all) affect the effectiveness of the Naive Bayes classifier? Explain why, or why not.
6. Naive Bayes is said to elegantly handle missing attribute values. For the datasets with missing values, is there any evidence that the performance is different on the instances with missing values, compared to the instances where all of the values are present? Does it matter which, or how many values are missing? Would a imputation strategy have any effect on this?

Don't forget that groups of 1 student should respond to question (1), and one other question of your choosing. Groups of 2 students should respond to question (1) and question (2), and two other questions of your choosing. Your responses should be about 150-250 words each.